In [1]:
# https://freeprog.tistory.com/329

import sys
import IPython
import imutils
import cv2
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *

from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import argparse


QApplication.setAttribute(Qt.AA_EnableHighDpiScaling, True)



class Left(QWidget): 
    
    def __init__(self, parent = None):

        super().__init__()
        self.setWindowTitle("My")
        self.setGeometry(900,500,w+100,h)
        
         
        
        
        
        
        
        
        # 전체 폼 박스
        formbox = QHBoxLayout()
        self.setLayout(formbox)

        # 좌, 우 레이아웃박스
        left = QVBoxLayout()
        middle = QVBoxLayout()

        # 그룹박스1 생성 및 좌 레이아웃 배치
        gb = QGroupBox('그리기 종류')        
        left.addWidget(gb)

        # 그룹박스1 에서 사용할 레이아웃
        box = QVBoxLayout()
        gb.setLayout(box)        

        # 그룹박스 1 의 라디오 버튼 배치
        text = ['horizon']
        self.radiobtns = []

        for i in range(len(text)):
            self.radiobtns.append(QRadioButton(text[i], self))
            self.radiobtns[i].clicked.connect(self.radioClicked)
            box.addWidget(self.radiobtns[i])

        self.radiobtns[0].setChecked(True)
        self.drawType = 0
         
        # 그룹박스2
        gb = QGroupBox('펜 설정')        
        left.addWidget(gb)        

        grid = QGridLayout()      
        gb.setLayout(grid)        

        label = QLabel('선굵기')
        grid.addWidget(label, 0, 0)

        self.combo = QComboBox()
        grid.addWidget(self.combo, 0, 1)       

        for i in range(1, 21):
            self.combo.addItem(str(i))

        label = QLabel('선색상')
        grid.addWidget(label, 1,0)        
         
        self.pencolor = QColor(0,0,0)
        self.penbtn = QPushButton()        
        self.penbtn.setStyleSheet('background-color: rgb(0,0,0)')
        self.penbtn.clicked.connect(self.showColorDlg)
        grid.addWidget(self.penbtn,1, 1)
         

        # 그룹박스3
        gb = QGroupBox('붓 설정')        
        left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)

        label = QLabel('붓색상')
        hbox.addWidget(label)                

        self.brushcolor = QColor(255,255,255)
        self.brushbtn = QPushButton()        
        self.brushbtn.setStyleSheet('background-color: rgb(255,255,255)')
        self.brushbtn.clicked.connect(self.showColorDlg)
        hbox.addWidget(self.brushbtn)
        
        # 그룹박스4
        gb = QGroupBox('크기 측정')        
        left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)
        
        self.pixelpntbtn = QPushButton()  
        self.pixelpntbtn.clicked.connect(self.pixelpnt)
        
        hbox.addWidget(self.pixelpntbtn)
        self.pixelpntbtn.setText('print')
        
        self.lbl = QLabel("  ")
        hbox.addWidget(self.lbl)
            
        # 그룹박스5
        gb = QGroupBox('불러오기')        
        left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)    
            
        self.pushButton = QPushButton("File Open")
        self.pushButton.clicked.connect(self.pushButtonClicked)
        self.label = QLabel()

        hbox.addWidget(self.pushButton)
        hbox.addWidget(self.label)
        
    
        
        
        left.addStretch(1)
        
        
        
        # 중간 레이아웃 박스에 그래픽 뷰 추가
        self.Graview()
        
        middle.addWidget(self.view)
         # 전체 폼박스에 좌우 박스 배치
        formbox.addLayout(left)
        formbox.addLayout(middle)


        formbox.setStretchFactor(left, 0)
        formbox.setStretchFactor(middle, 0)
        
        
        
        
        
        
    def Graview(self):
        global w,h
        self.view =CView(self)   
        
        self.view.setFixedHeight(h)
        self.view.setFixedWidth(w)

                
        

        
        
        
        
        
        
         
    def radioClicked(self):
        for i in range(len(self.radiobtns)):
            if self.radiobtns[i].isChecked():
                self.drawType = i                
                break

    def checkClicked(self):
        pass
             
    def showColorDlg(self):       
         
        # 색상 대화상자 생성      
        color = QColorDialog.getColor()

        sender = self.sender()

        # 색상이 유효한 값이면 참, QFrame에 색 적용
        if sender == self.penbtn and color.isValid():           
            self.pencolor = color
            self.penbtn.setStyleSheet('background-color: {}'.format( color.name()))
        else:
            self.brushcolor = color
            self.brushbtn.setStyleSheet('background-color: {}'.format( color.name()))

    def pixelpnt(self):
        global dy
        global pixelsPerMetric
        print(pixelsPerMetric)
        
        self.size= dy/pixelsPerMetric
        
        self.txt=repr("{:.5f} ".format(self.size)+chr(956)+"m")     
        
        
        self.lbl.setText(self.txt)
        self.lbl.adjustSize()    

    def pushButtonClicked(self):
        import os
        
        self.tuplefname = QFileDialog.getOpenFileName(self)
        self.fname = self.tuplefname[0]
        dir,file = os.path.split(self.fname)
        self.label.setText(file)
        self.fname = os.path.basename(self.fname)
        print(os.getcwd())
        print(dir)


        
# QGraphicsView display QGraphicsScene
class CView(QGraphicsView):
    
    
    def __init__(self, parent):
        
        super().__init__(parent)       
        self.scene = QGraphicsScene()

        pic = QPixmap(fname)
        self.scene.addPixmap(pic)
        self.setScene(self.scene)
        
        self.items = []
        
        self.start = QPointF()
        self.end = QPointF()
        
        self.setRenderHint(QPainter.HighQualityAntialiasing)
        
    def moveEvent(self, e): # 없애면 스크롤바 생김
        rect = QRectF(self.rect())
        rect.adjust(0,0,-2,-2)

        self.scene.setSceneRect(rect)
    
    def mousePressEvent(self, e):
        
        if e.buttons() & Qt.RightButton:           
            if len(self.items) > 0:
                self.scene.removeItem(self.items[-1])
                del(self.items[-1])

        if e.button() == Qt.LeftButton:
            # 시작점 저장
            self.start = e.pos()
            self.end = e.pos() 
            
    
    def mouseMoveEvent(self, e):  
        
        if e.buttons() & Qt.RightButton:           
            if len(self.items) > 0:
                self.scene.removeItem(self.items[-1])
                del(self.items[-1])
                
        # e.buttons()는 정수형 값을 리턴, e.button()은 move시 Qt.Nobutton 리턴 
        if e.buttons() & Qt.LeftButton:           

            self.end = e.pos()

            pen = QPen(self.parent().pencolor, self.parent().combo.currentIndex())

        
            # 수평선 그리기
            if self.parent().drawType == 0:
                 
                # 장면에 그려진 이전 선을 제거            
                if len(self.items) > 0:
                    self.scene.removeItem(self.items[-1])
                    del(self.items[-1])  
                    self.scene.removeItem(self.items[-1])
                    del(self.items[-1])  
                
                hori1 = QLineF(0, self.start.y(), w, self.start.y())
                
                self.items.append(self.scene.addLine(hori1, pen))
                
                hori2 = QLineF(self.end.x(), self.start.y(), self.end.x(), self.end.y())
                
                self.items.append(self.scene.addLine(hori2, pen))
                print(self.start.y(),self.end.y())
                
                
                
                """
                # 현재 선 추가
                hori = QLineF(self.start.x(), self.start.y(), self.end.x(), self.start.y())                
                self.items.append(self.scene.addLine(hori, pen))
                """

                
                
                
                
            
    
    def mouseReleaseEvent(self, e):  
        
        if e.button() == Qt.RightButton:
            if len(self.items) > 0:
                self.scene.removeItem(self.items[-1])
                #del(self.items[-1])    

        if e.button() == Qt.LeftButton:

            pen = QPen(self.parent().pencolor, self.parent().combo.currentIndex())
            global dy 
            dy = abs(self.start.y()-self.end.y())
            self.dx = abs(self.start.x()-self.end.x())
            print(dy, self.dx)
            

    """
            if self.parent().drawType == 0:
                
                self.items.clear()
                hori = QLineF(0, self.start.y(), w, self.start.y())
                self.scene.addLine(hori, pen)
                hori = QLineF(self.end.x(), self.start.y(), self.end.x(), self.end.y())
                self.scene.addLine(hori, pen)
                print(self.start,self.end)
    """  


def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

   
global fname
fname = './images/example_04.bmp'
width = 10.0



# load the image, convert it to grayscale, and blur it slightly
image =cv2.imread(fname)
cv2.imshow('image',image)
h, w = image.shape[:2]
cv2.waitKey(0)
image = image[930:990,0:]
cv2.imshow('croped',image)
cv2.waitKey(0)
cv2.destroyAllWindows() 

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
"""
cv2.imshow('Gray', gray)
cv2.waitKey(0)
cv2.destroyAllWindows()
"""
blur = cv2.GaussianBlur(gray, (7, 7), 0)

cv2.imshow('blur', blur)
cv2.waitKey(0)
cv2.destroyAllWindows()

# perform edge detection, then perform a dilation + erosion to
# close gaps in between object edges
canny = cv2.Canny(blur, 50, 100)
edged = cv2.dilate(canny, None, iterations=1)
edged = cv2.erode(edged, None, iterations=1)

# find contours in the edge map
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

# sort the contours from left-to-right and initialize the
# 'pixels per metric' calibration variable
(cnts, _) = contours.sort_contours(cnts)
global pixelsPerMetric
pixelsPerMetric = None

# loop over the contours individually
for c in cnts:
    # if the contour is not sufficiently large, ignore it
    if cv2.contourArea(c) < 100:
        continue

    # compute the rotated bounding box of the contour
    orig = image.copy()
    box = cv2.minAreaRect(c)
    box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
    box = np.array(box, dtype="int")
    #print(box)

    # order the points in the contour such that they appear
    # in top-left, top-right, bottom-right, and bottom-left
    # order, then draw the outline of the rotated bounding
    # box
    box = perspective.order_points(box)
    cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)

    # loop over the original points and draw them
    for (x, y) in box:
        cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255), -1)

    # unpack the ordered bounding box, then compute the midpoint
    # between the top-left and top-right coordinates, followed by
    # the midpoint between bottom-left and bottom-right coordinates
    (tl, tr, br, bl) = box
    (tltrX, tltrY) = midpoint(tl, tr)
    (blbrX, blbrY) = midpoint(bl, br)

    # compute the midpoint between the top-left and top-right points,
    # followed by the midpoint between the top-righ and bottom-right
    (tlblX, tlblY) = midpoint(tl, bl)
    (trbrX, trbrY) = midpoint(tr, br)

    # draw the midpoints on the image
    cv2.circle(orig, (int(tltrX), int(tltrY)), 5, (255, 0, 0), -1)
    cv2.circle(orig, (int(blbrX), int(blbrY)), 5, (255, 0, 0), -1)
    cv2.circle(orig, (int(tlblX), int(tlblY)), 5, (255, 0, 0), -1)
    cv2.circle(orig, (int(trbrX), int(trbrY)), 5, (255, 0, 0), -1)

    # draw lines between the midpoints
    cv2.line(orig, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)),
        (255, 0, 255), 2)
    cv2.line(orig, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)),
        (255, 0, 255), 2)

    # compute the Euclidean distance between the midpoints
    dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
    dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))

    # if the pixels per metric has not been initialized, then
    # compute it as the ratio of pixels to supplied metric
    # (in this case, inches)
    if pixelsPerMetric is None:
        pixelsPerMetric = dB / width

    # compute the size of the object
    dimA = dA / pixelsPerMetric
    dimB = dB / pixelsPerMetric
    
    # draw the object sizes on the image
    cv2.putText(orig, "{:.5f}".format(dimB),
        (int(tltrX - 15), int(tltrY - 10)), cv2.FONT_HERSHEY_SIMPLEX,0.65, (255, 255, 255), 2)
    cv2.putText(orig, "{:.5f}".format(dimA),(int(trbrX + 10), int(trbrY)),
        cv2.FONT_HERSHEY_SIMPLEX,0.65, (255, 255, 255), 2)   


    # show the output image
    cv2.imshow("Image", orig)
    cv2.waitKey(0)
cv2.destroyAllWindows()



if __name__=="__main__":
    app = QApplication(sys.argv)
    ex = Left()
    ex.show()
    app.exec_()
    
app = IPython.Application.instance()
app.kernel.do_shutdown(True)
app.kernel.start()
